# Variable Autoencoder

## Objectives


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os

os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import layers

import numpy as np

from modules.dataset import OneHotProtDataset
from modules.layers import SamplingLayer, VariationalLossLayer

from sklearn.model_selection import train_test_split

import sys

In [10]:
run_id = "00"
run_dir = "./run/vae/"
datasets_dir = "../data/SAbDab"

## Parameters


In [11]:
latent_dim = 2
loss_weights = [1, 0.06]

scale = 0.2
seed = 123

batch_size = 64
epochs = 10
fit_verbosity = 1

## Prepare data

In [12]:
x_data, y_data, vector_size, alphabet_size = OneHotProtDataset.get_data()

In [13]:
x_train, y_train, x_test, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

In [14]:
inputs = keras.Input(shape=(38, 38, 1))
x = layers.Conv2D(32, 3, strides=1, padding="same", activation="relu")(inputs)
x = layers.Conv2D(64, 3, strides=2, padding="same", activation="relu")(x)
x = layers.Conv2D(64, 3, strides=2, padding="same", activation="relu")(x)
x = layers.Conv2D(64, 3, strides=1, padding="same", activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = SamplingLayer()([z_mean, z_log_var])

encoder = keras.Model(inputs, [z_mean, z_log_var, z], name="encoder")
# encoder.summary()

TypeError: Exception encountered when calling layer "sampling_layer_1" (type SamplingLayer).

in user code:

    File "/media/data/Documents/Projects/hackathon/D4GEN/2024/intelligent_antibodies/notebooks/modules/layers/SamplingLayer.py", line 19, in call  *
        epsilon = Normal(0, 1).sample((batch_size, latent_dim)).to(z_mean.device)
    File "/home/sortion/.local/share/miniforge3/envs/intelligent-antibodies/lib/python3.11/site-packages/torch/distributions/normal.py", line 68, in sample  *
        shape = self._extended_shape(sample_shape)
    File "/home/sortion/.local/share/miniforge3/envs/intelligent-antibodies/lib/python3.11/site-packages/torch/distributions/distribution.py", line 267, in _extended_shape  *
        sample_shape = torch.Size(sample_shape)

    TypeError: torch.Size() takes an iterable of 'int' (item 0 is 'NoneType')


Call arguments received by layer "sampling_layer_1" (type SamplingLayer):
  • inputs=['tf.Tensor(shape=(None, 2), dtype=float32)', 'tf.Tensor(shap

## References

- <https://keras.io/examples/generative/vae/>
- <https://fidle.cnrs.fr>